# Background

详见 [F:\OPPO_华勤_OTS\record.md](F:/OPPO_华勤_OTS/record.md)

In [1]:
from mxESP32Debugger.debugger import Dbg
from tqdm import trange
import time
import toml
from loguru import logger

In [2]:
dev = Dbg()
dev.power_init()
dev.usb_config()

2024-12-23 20:37:41.790 | INFO     | mxESP32Debugger.debugger:connect:248 - Using serial port to connect. Port: COM17
2024-12-23 20:37:41.791 | INFO     | mxESP32Debugger.debugger:connect:253 - Using ESP32-S3R8.
2024-12-23 20:37:41.791 | INFO     | mxESP32Debugger.debugger:connect:254 - Embedded software version: v1.0
2024-12-23 20:37:41.792 | INFO     | mxESP32Debugger.debugger:connect:255 - Library version: v241210.0.rc3.
2024-12-23 20:37:41.814 | INFO     | mxESP32Debugger.debugger:get_extboard_version:1659 - Extension board version: v0.1


(True, None)

# Configure

In [3]:
config = toml.load('./mot6016_1.toml')

reg_list_1 = config['init']['reg_list_1']
reg_list_2 = config['init']['reg_list_2']
reg_list_check = config['init']['check']

def ots_init():
    for reg in reg_list_1:
        ret, data = dev.i2c_write_read(0x74, reg, 0)
        if ret is False:
            logger.error(f'Write {reg} failed.')
            return False
    
    ret, data = dev.i2c_write_read(0x74, [0x00], 2)
    if ret == False:
        logger.error("Failed to read chipid")
        return False
    
    chipid = data[0] << 8 | data[1]
    if chipid != 0x585B:
        logger.error(f"Failed to read chipid: {chipid}")
        return False
    
    for reg in reg_list_2:
        ret, data = dev.i2c_write_read(0x74, reg, 0)
        if ret is False:
            logger.error(f'Write {reg} failed.')
            return False
        
    for index, reg in enumerate(reg_list_check):
        ret, data = dev.i2c_write_read(0x74, [reg[0]], 1)
        if ret is False:
            logger.error(f'Read 0x{reg[0]:02X} failed.')
            return False
        
        if data[0] != reg[1]:
            logger.error(f"Failed to read {reg}")
            return False
        
    return True

In [4]:
LDO_EN_PIN_1 = 39
LDO_EN_PIN_2 = 38
ERROR_TRIGGER_PIN = 36
LDO_EN_PIN_3 = 37

VDD_PIN = LDO_EN_PIN_1
I2C_PIN = LDO_EN_PIN_2
LDP_PIN = LDO_EN_PIN_3
dev.gpio_config(pin=VDD_PIN, mode=0x02, pull_down=True, pull_up=False)
dev.gpio_config(pin=I2C_PIN, mode=0x02, pull_down=True, pull_up=False)
dev.gpio_config(pin=LDP_PIN, mode=0x02, pull_down=True, pull_up=False)
dev.gpio_config(pin=ERROR_TRIGGER_PIN, mode=0x02, pull_down=True, pull_up=False)
dev.gpio_write_read(pin=VDD_PIN, level=0)
dev.gpio_write_read(pin=I2C_PIN, level=0)
dev.gpio_write_read(pin=LDP_PIN, level=0)
dev.gpio_write_read(pin=ERROR_TRIGGER_PIN, level=0)

(True, None)

In [ ]:
failed_times = 0

dev.gpio_write_read(pin=VDD_PIN, level=1)  # VDD3 / AVDD 

for i in trange(50000, desc='Power ON/OFF Test'):
    dev.gpio_write_read(ERROR_TRIGGER_PIN, 0)
    
    dev.gpio_write_read(pin=LDP_PIN, level=1)  # VLD
    dev.gpio_write_read(pin=VDD_PIN, level=1)  # VDD3 / AVDD 
    time.sleep(100/1000)
    
    dev.gpio_write_read(pin=LDP_PIN, level=0)  # VLD
    time.sleep(165/1000)
    dev.gpio_write_read(pin=VDD_PIN, level=0)  # VDD3 / AVDD 
    dev.gpio_write_read(pin=VDD_PIN, level=1)  # VDD3 / AVDD 
    
    time.sleep(11)
    dev.gpio_write_read(pin=LDP_PIN, level=1)  # VLD
    time.sleep(20/1000)
    ret, data = dev.i2c_write_read(0x74, [0x00], 1)
    assert data[0] == 0x30, "no 0x74 device found"
    
    ret, data = dev.i2c_write_read(0x74, [0x00], 2)
    if ret is False:
        logger.error("try to read 0x00 register failed.")
        dev.gpio_write_read(ERROR_TRIGGER_PIN, 1)
    else:
        if data[0] not in [0x30, 0x58]:
            logger.error(f"read 0x00 for 0x30 failed. 0x{data[0]:02X}")
            dev.gpio_write_read(ERROR_TRIGGER_PIN, 1)
        else:
            dev.i2c_write_read(0x74, [0x06, 0x17], 0)  # Exit PD
            
            # ret, data = dev.i2c_write_read(0x74, [0x00], 1)
            # assert data[0] == 0x30, f'Read data 0x{data[0]:02X} from 0x00 failed. ({i})'
            if ots_init() is False:
                dev.gpio_write_read(ERROR_TRIGGER_PIN, 1)
                failed_times += 1
                
            dev.i2c_write_read(0x74, [0x06, 0x1F], 0)  # Enter PD
    

Power ON/OFF Test:   0%|          | 89/50000 [17:27<163:14:21, 11.77s/it]